# Regressione non Lineare (parte 2)

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Setup

- Importare i package necessari

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

- In Jupyter, configurare l'output di matplotlib integrato nel notebook

In [2]:
%matplotlib inline

## Caso di studio: Predizione dei prezzi delle case

- Riprendiamo dalla scorsa esercitazione il dataset relativo ai prezzi delle case
- Forniamo tale dataset all'URL https://git.io/fjGjx già adattato per essere caricato con `read_csv` con le opzioni di default

In [3]:
import os.path
if not os.path.exists("housing.csv"):
    from urllib.request import urlretrieve
    urlretrieve("https://git.io/fjGjx", "housing.csv")

In [4]:
housing = pd.read_csv("housing.csv")

### Lista delle variabili

- CRIM: tasso di criminalità pro capite per zona
- ZN: proporzione terreno residenziale per lotti maggiori di 25.000 piedi quadrati (circa 2300 m2)
- INDUS: proporzione di acri industriali non commerciali per città
- CHAS: variabile fittizia Charles River, 1 se il tratto affianca il fiume, altrimenti 0
- NOX: concentrazione di ossido d’azoto (parti per 10 milioni)
- RM: numero medio di stanze per abitazione
- AGE: proporzione delle unità abitate costruite prima del 1940
- DIS: distanze pesate verso i cinque uffici di collocamento di Boston
- RAD: indice di accessibilità rispetto alle grandi vie radiali di comunicazione
- TAX: tasso di imposte sulla casa per 10.000 dollari
- PTRATIO: rapporto allievi-docenti per città
- B: 1000(Bk - 0.63)2, dove Bk è la proporzione di persone di origine afroamericana
- LSTAT: percentuale di popolazione con basso reddito
- **MEDV: valore mediano delle abitazioni di proprietà in migliaia di dollari**
  - vogliamo stimare il valore di questa variabile in funzione delle altre

In [5]:
housing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


- Estraiamo dal frame
  - la serie `y` con i valori della variabile `MEDV` da prevedere
  - il frame `X` con i valori di tutte le altre variabili, utilizzabili per la predizione

In [6]:
y = housing["MEDV"]
X = housing.drop(columns="MEDV")

## Divisione tra training e validation set

- Dividiamo i dati caricati casualmente in
  - un _training set_ per addestrare i modelli
  - un _validation set_ disgiunto su cui verificare l'accuratezza del modello
- Usiamo la funzione `train_test_split` di scikit-learn
  - con `test_size` indichiamo quanti dati vanno nel validation set, i restanti andranno nel training set
  - con `random_state` fissiamo un seed per la suddivisione casuale
  - la funzione mescola i dati di `X` e `y` in modo congiunto, mantenendo la corrispondenza esistente tra le posizioni dei dati

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = \
    train_test_split(X, y, test_size=1/3, random_state=42)

## Funzioni per la validazione

- Riprendiamo dalla prima parte le funzioni per estrarre e stampare le misure di valutazione del modello (MSE, errore relativo, R²)

In [8]:
# importo MSE e R²
from sklearn.metrics import mean_squared_error, r2_score

# definisco funzione per errore relativo
def relative_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

# funzione per calcolare e stampare tutte e tre
def print_eval(X, y, model):
    preds = model.predict(X)
    print("   Mean squared error: {:.5}".format(mean_squared_error(y, preds)))
    print("       Relative error: {:.5%}".format(relative_error(y, preds)))
    print("R-squared coefficient: {:.5}".format(r2_score(y, preds)))

## Esercizio 1: Ripasso modelli

Addestrare sul training set creato sopra tre modelli diversi con le seguenti configurazioni, per ciascuno stampare le misure di valutazione sul validation set e i coefficienti associati a ciascuna variabile

- **(1a)** regressione lineare semplice
- **(1b)** regressione lineare con regolarizzazione L2 (regressione ridge), assumendo $\alpha=1$
- **(1c)** regressione lineare con standardizzazione delle feature

Infine:

- **(1d)** visualizzare in un unico frame i coefficienti di tutti e tre i modelli (una riga per variabile, una colonna per modello)

In [9]:
# sono eseguiti quì tutti gli import necessari da scikit-learn
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

- In tutti e tre i modelli, dai segni dei coefficienti possiamo vedere quali fenomeni influiscono positivamente e negativamente sul prezzo
  - ad es. il prezzo delle case è più alto se vicine al fiume (`CHAS`), mentre decresce con la criminalità (`CRIM`)
- Nella regressione ridge i valori assoluti più alti sono ridotti (es. `NOX` e `RM`)
- Con la standardizzazione delle feature otteniamo valori su scale simili, che possiamo confrontare alla pari
  - ad es. negli altri modelli il coefficiente di `NOX` è alto perché i valori di tale variabile sono bassi (la media è circa 0.55, contro quelle superiori a 3 delle altre variabili)
  - nel modello con standardizzazione assumono invece più peso il numero di stanze (`RM`) e la distanza dagli uffici di collocamento (`DIS`)

## Regressione Lasso

- La regolarizzazione L2 impedisce che i parametri del modello assumano valori troppo alti
- I valori dei parametri sono comunque tutti non nulli, tutte le variabili vengono coinvolte nella predizione
- Vorremmo addestrare un modello meno complesso, dove alcuni parametri hanno valori nulli, **ignorando completamente le variabili meno rilevanti**
  - ad es. variabili con valori dipendenti da altre (_multicollinearità_)
- Questo si può ottenere tramite la regolarizzazione L1, basata sulla norma 1, definita su un vettore $\mathbf{x}$ di $n$ elementi come
$$ \left\Vert\mathbf{x}\right\Vert_1 = \sum_{i=1}^n{\left\vert x_i\right\vert} = \left\vert x_1\right\vert+\ldots+\left\vert x_n\right\vert $$

- La regressione _lasso_ consiste nella regressione lineare con regolarizzazione L1, basata quindi sul minimizzare la funzione d'errore
$$ E = \frac{1}{2m}\left\Vert\mathbf{X}\mathbf{\theta}-\mathbf{y}\right\Vert_2^2 + \alpha\left\Vert\mathbf{\theta}\right\Vert_1 $$
- Come per la regressione ridge, il parametro $\alpha$ controlla il peso della regolarizzazione
- La regressione lasso si esegue usando un modello `Lasso`, su cui possiamo impostare come in `Ridge` il parametro `alpha`

In [18]:
from sklearn.linear_model import Lasso
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr", Lasso(alpha=1))
])
model.fit(X_train, y_train)

Pipeline(steps=[('scale', StandardScaler()), ('regr', Lasso(alpha=1))])

- Vediamo i coefficienti del modello risultante

In [19]:
pd.Series(model.named_steps["regr"].coef_, X.columns)

CRIM      -0.000000
ZN         0.000000
INDUS     -0.000000
CHAS       0.270752
NOX       -0.000000
RM         2.641503
AGE       -0.000000
DIS       -0.000000
RAD       -0.000000
TAX       -0.000000
PTRATIO   -1.200172
B          0.311046
LSTAT     -3.814854
dtype: float64

- La regolarizzazione L1 ha contribuito ad annullare quanti più coefficienti possibile, creando un modello che considera solo 5 variabili
- Ma qual'è l'accuratezza di tale modello?

In [20]:
print_eval(X_val, y_val, model)

   Mean squared error: 25.984
       Relative error: 19.41638%
R-squared coefficient: 0.65457


- L'accuratezza è peggiore rispetto ai casi precedenti: in questo caso la regolarizzazione è stata quindi eccessiva

- Cosa succede diminuendo il parametro `alpha`, ovvero il peso della regolarizzazione?

In [21]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr", Lasso(alpha=0.2)) # <-- cambiato da 1
])
model.fit(X_train, y_train)

Pipeline(steps=[('scale', StandardScaler()), ('regr', Lasso(alpha=0.2))])

In [22]:
pd.Series(model.named_steps["regr"].coef_, X.columns)

CRIM      -0.394705
ZN         0.192948
INDUS     -0.000000
CHAS       0.827011
NOX       -0.812358
RM         2.944153
AGE       -0.000000
DIS       -1.514910
RAD        0.000000
TAX       -0.000000
PTRATIO   -1.698942
B          0.842742
LSTAT     -4.009724
dtype: float64

- I coefficienti non nulli sono aumentati da 5 a 11

In [23]:
print_eval(X_val, y_val, model)

   Mean squared error: 22.527
       Relative error: 16.62391%
R-squared coefficient: 0.70053


- L'accuratezza è di poco inferiore a quella ottenuta con gli altri modelli
- Questo modello richiede però solo 9 variabili invece di 13

## Elastic Net

- La regressione _elastic net_ combina insieme le regolarizzazioni L2 e L1 usate in ridge e lasso
- Si applica in scikit-learn tramite la classe `ElasticNet`, per cui l'errore è calcolato come:
$$ E = \underbrace{\frac{1}{2m} ||X\theta - y||_2 ^ 2}_{\text{errore sui dati}} + \underbrace{\alpha \rho ||\theta||_1}_{\text{L1}} + \underbrace{\frac{\alpha(1-\rho)}{2} ||\theta||_2 ^ 2}_{\text{L2}} $$
- I parametri impostabili sono
  - `alpha` ($\alpha$) che determina il peso generale della regolarizzazione
  - `l1_ratio` ($\rho$, compreso tra 0 e 1) che determina il peso di L1 relativo al totale (con $\rho=1$ si ha la regressione lasso, con $\rho=0$ la ridge)

In [24]:
from sklearn.linear_model import ElasticNet
model = Pipeline([
    ("scale",  StandardScaler()),
    ("regr", ElasticNet(alpha=0.2, l1_ratio=0.1))
])
model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

   Mean squared error: 22.092
       Relative error: 16.18298%
R-squared coefficient: 0.70631


## Esercizio 2: Elastic Net con pesi separati

- **(2a)** Definire una funzione `elastic_net_with_alphas` che restituisca un modello elastic net (non addestrato) con pesi dati separatamente per la regolarizzazione L2 e L1
  - si ricordi che il parametro `alpha` è la somma dei due pesi
- **(2b)** Servendosi di tale funzione, addestrare e validare un modello elastic net con $\alpha_{L2}=1, \alpha_{L1}=0.1$ e standardizzazione delle feature

In [26]:
def elastic_net_with_alphas(alpha_l2, alpha_l1):
    ...

## Ripasso: Regressione polinomiale univariata

- Abbiamo visto in precedenza la regressione polinomiale su una sola variabile $X$ (univariata), corrispondente alla regressione lineare sulle variabili $X,X^2,X^3,\ldots$
- Per generare queste variabili utilizziamo il filtro `PolynomialFeatures`

In [29]:
from sklearn.preprocessing import PolynomialFeatures

- Siano date ad esempio due osservazioni di una variabile...

In [30]:
sample = np.array([ [ 2],
                    [-3] ])

- Possiamo ottenere ad es. le potenze fino al 4° grado
  - `include_bias=True` specifica di non includere il termine di grado 0

In [31]:
poly = PolynomialFeatures(degree=4, include_bias=False)
poly.fit_transform(sample)
#         X   X^2   X^3   X^4

array([[  2.,   4.,   8.,  16.],
       [ -3.,   9., -27.,  81.]])

## Regressione polinomiale multivariata

- In presenza di più di una variabile, la regressione polinomiale genera tutti i possibili termini fino al grado impostato, includendo anche **termini basati su più variabili**
- Vediamo un esempio con 2 generiche variabili $A$ e $B$

In [32]:
#                     A   B
sample = np.array([ [ 2, -3],
                    [ 4, -5] ])

- Applicando il filtro `PolynomialFeatures` con grado 2...

In [33]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly.fit_transform(sample)
#         A     B    A^2   A*B   B^2

array([[  2.,  -3.,   4.,  -6.,   9.],
       [  4.,  -5.,  16., -20.,  25.]])

- Le variabili generate (escludendo il grado nullo) sono 5: $A,B,A^2,AB,B^2$
- Oltre ai quadrati delle singole variabili abbiamo quindi anche i prodotti tra di esse

- Possiamo usare il metodo `get_feature_names` del filtro per avere una lista delle variabili calcolate
  - il filtro deve già essere stato "addestrato" con `fit` o `fit_transform`
  - è possibile passare una lista di nomi delle variabili originali, altrimenti sono usati `x0`, `x1`, ...

In [34]:
poly.get_feature_names()

['x0', 'x1', 'x0^2', 'x0 x1', 'x1^2']

In [35]:
poly.get_feature_names(["A", "B"])

['A', 'B', 'A^2', 'A B', 'B^2']

- Aumentando il grado massimo, le variabili generate **aumentano rapidamente**
- Ad esempio, aumentando il grado da 2 a 3...

In [36]:
poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit_transform(sample)

array([[   2.,   -3.,    4.,   -6.,    9.,    8.,  -12.,   18.,  -27.],
       [   4.,   -5.,   16.,  -20.,   25.,   64.,  -80.,  100., -125.]])

- ...generiamo 9 variabili, ovvero:

In [37]:
poly.get_feature_names(["A", "B"])

['A', 'B', 'A^2', 'A B', 'B^2', 'A^3', 'A^2 B', 'A B^2', 'B^3']

- Cosa succede con un numero iniziale di variabili più alto?
- Selezioniamo ad esempio dalle variabili X del dataset le 5 feature che avevano coefficiente non nullo

In [38]:
# la lista delle feature da considerare è:
Xsub_feats = ["CHAS", "RM", "PTRATIO", "B", "LSTAT"]
# creo una selezione sia dal training che dal validation set
Xsub_train = X_train[Xsub_feats]
Xsub_val = X_val[Xsub_feats]
# stampo il numero di colonne
Xsub_train.shape[1]

5

- Generando le feature polinomiali con grado massimo 2...

In [39]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly.fit_transform(Xsub_train).shape[1]

20

- ...otteniamo 20 feature distinte!
- Le feature includono infatti tutte le possibili coppie di variabili, oltre ai quadrati di ciascuna

In [40]:
poly.get_feature_names(Xsub_train.columns)

['CHAS',
 'RM',
 'PTRATIO',
 'B',
 'LSTAT',
 'CHAS^2',
 'CHAS RM',
 'CHAS PTRATIO',
 'CHAS B',
 'CHAS LSTAT',
 'RM^2',
 'RM PTRATIO',
 'RM B',
 'RM LSTAT',
 'PTRATIO^2',
 'PTRATIO B',
 'PTRATIO LSTAT',
 'B^2',
 'B LSTAT',
 'LSTAT^2']

- Aumentando ulteriormente il grado, il numero di variabili cresce esponenzialmente

In [41]:
poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit_transform(Xsub_train).shape[1]

55

In [42]:
poly = PolynomialFeatures(degree=4, include_bias=False)
poly.fit_transform(Xsub_train).shape[1]

125

In [43]:
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit_transform(Xsub_train).shape[1]

251

In [44]:
poly = PolynomialFeatures(degree=6, include_bias=False)
poly.fit_transform(Xsub_train).shape[1]

461

In [45]:
poly = PolynomialFeatures(degree=7, include_bias=False)
poly.fit_transform(Xsub_train).shape[1]

791

In [46]:
poly = PolynomialFeatures(degree=8, include_bias=False)
poly.fit_transform(Xsub_train).shape[1]

1286

- Questa crescita è ancora più evidente con la matrice completa `X_train`, con 13 variabili

In [47]:
X_train.shape[1]

13

- Generando le feature polinomiali con grado massimo 2 otteniamo 104 feature

In [48]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly.fit_transform(X_train).shape[1]

104

- Aumentando ulteriormente il grado, il numero di variabili cresce enormemente
  - con grado 10 si supera il milione di variabili

In [49]:
poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit_transform(X_train).shape[1]

559

In [50]:
poly = PolynomialFeatures(degree=4, include_bias=False)
poly.fit_transform(X_train).shape[1]

2379

In [51]:
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit_transform(X_train).shape[1]

8567

In [52]:
poly = PolynomialFeatures(degree=6, include_bias=False)
poly.fit_transform(X_train).shape[1]

27131

- All'aumentare delle variabili, aumenta il tempo necessario per l'addestramento del modello
- Prendiamo ad esempio come riferimento un modello ElasticNet polinomiale con standardizzazione delle feature generate

In [53]:
def poly_std_elasticnet(degree):
    return Pipeline([
        ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
        ("std",  StandardScaler()),
        ("regr", ElasticNet(alpha=0.5, l1_ratio=0.2))
    ])

- Eseguiamo la prova su un modello di grado 2 sia col sottoinsieme di 5 feature indicato sopra che con tutte le feature
- Usiamo il comando "magico" `%time` per riportare in output il tempo di esecuzione

In [54]:
model = poly_std_elasticnet(2)
%time model.fit(Xsub_train, y_train)
print_eval(Xsub_val, y_val, model)

CPU times: user 3.29 ms, sys: 2.13 ms, total: 5.42 ms
Wall time: 4.59 ms
   Mean squared error: 22.74
       Relative error: 17.05512%
R-squared coefficient: 0.69769


In [55]:
model = poly_std_elasticnet(2)
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 27.1 ms, sys: 66.2 ms, total: 93.3 ms
Wall time: 35.9 ms
   Mean squared error: 19.854
       Relative error: 15.44921%
R-squared coefficient: 0.73606


- L'addestramento richiede una frazione di secondo, anche per via del numero limitato di istanze

- Verifichiamo ora cosa accade in entrambi i casi con grado 5

In [56]:
model = poly_std_elasticnet(5)
%time model.fit(Xsub_train, y_train)
print_eval(Xsub_val, y_val, model)

CPU times: user 164 ms, sys: 261 ms, total: 425 ms
Wall time: 134 ms
   Mean squared error: 16.714
       Relative error: 14.62323%
R-squared coefficient: 0.7778


In [57]:
model = poly_std_elasticnet(5)
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 2.02 s, sys: 436 ms, total: 2.46 s
Wall time: 1.98 s
   Mean squared error: 11.969
       Relative error: 11.63031%
R-squared coefficient: 0.84089


- L'accuratezza del modello migliora sensibilmente, ma **con tempi di addestramento molto superiori**
  - più di 10 volte superiori con 5 feature
  - più di 100 volte superiori con 13 feature
  - con dataset più grandi, avremmo tempi di addestramento insostenibili

## Regressione con funzioni kernel

- Nella regressione polinomiale si eseguono prodotti tra dati con dimensioni aggiunte e rappresentate esplicitamente
- Le _funzioni kernel_ permettono di calcolare gli stessi prodotti senza calcolare esplicitamente le dimensioni aggiunte
- Questo permette di ottenere **modelli non lineari senza l'aggiunta di variabili**
- Esistono diverse funzioni kernel utilizzabili con diversi parametri impostabili
- Ad esempio, il kernel polinomiale è definito dalla formula
$$ K(\mathbf{a},\mathbf{b}) = \left(\mathbf{a}\cdot\mathbf{b}+c\right)^d $$
  - $d$ e $c$ sono parametri del kernel, in particolare $d$ è il grado del polinomio

- La classe `KernelRidge` implementa la regressione ridge con l'applicazione di una funzione kernel
- Col parametro `kernel` si indica il tipo di kernel con una stringa, ad es. `"poly"` per un kernel polinomiale
- Ulteriori parametri riguardano il kernel, per quello polinomiale sono `degree` ($d$) e `coef0` ($c$)

In [58]:
from sklearn.kernel_ridge import KernelRidge
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="poly", degree=5))
])
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 33.4 ms, sys: 69 ms, total: 102 ms
Wall time: 33 ms
   Mean squared error: 14.765
       Relative error: 13.09973%
R-squared coefficient: 0.80371


- Abbiamo ottenuto un'accuratezza più elevata rispetto ai modelli lineari, ma in tempi molto più brevi rispetto alla regressione polinomiale

- Aumentando arbitrariamente il grado del polinomio, il tempo impiegato per l'addestramento non cambia
  - in questo caso comunque un grado alto non porta ad un modello accurato

In [59]:
from sklearn.kernel_ridge import KernelRidge
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="poly", degree=15))
])
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 32.6 ms, sys: 56.6 ms, total: 89.2 ms
Wall time: 26.2 ms
   Mean squared error: 2.0188e+07
       Relative error: 8417.85655%
R-squared coefficient: -2.6838e+05


- Va però ricordato che la complessità cresce quadraticamente col numero di istanze di training
- Ad esempio, addestrando un modello su _tutti_ i dati invece che sul solo training set, quindi sul 50\% di istanze in più...

In [60]:
from sklearn.kernel_ridge import KernelRidge
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="poly", degree=5))
])
%time model.fit(X, y)
print_eval(X_val, y_val, model)

CPU times: user 52.2 ms, sys: 154 ms, total: 206 ms
Wall time: 62.6 ms
   Mean squared error: 8.2617
       Relative error: 8.99352%
R-squared coefficient: 0.89017


- ... il tempo necessario è circa il doppio

- Possiamo testare anche funzioni kernel diverse, ad esempio RBF (_radial basis function_)
  - RBF ha valori tanto più elevati quanto più i valori X sono vicini a 0 (ovvero la media, usando dati standardizzati)
- La funzione RBF ha la forma di una gaussiana, di cui si può impostare l'ampiezza col parametro `gamma`

In [61]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="rbf", gamma=0.01))
])
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 29.7 ms, sys: 77 ms, total: 107 ms
Wall time: 32 ms
   Mean squared error: 43.906
       Relative error: 19.38425%
R-squared coefficient: 0.41632


- In questo caso specifico il kernel RBF non funziona bene tanto quanto il polinomiale

## k-Fold Cross Validation

- La _cross validation_ si riferisce in generale alla valutazione di un modello di predizione su dati differenti rispetto a quelli su cui è addestrato
- Finora l'abbiamo svolta usando il semplice metodo _hold-out_
  - i dati sono suddivisi casualmente in _training set_ e _validation set_ con proporzioni configurabili
  - un modello è addestrato sul training set e validato sul validation set
- Il metodo _k-fold_ è un'alternativa che permette una validazione più accurata
  - i dati sono divisi causalmente in k gruppi (_fold_)
  - ciascun gruppo è validato su un modello addestrato su tutti gli altri gruppi
  - i risultati dei singoli test sono aggregati
- scikit-learn fornisce un supporto generico per la cross-validation di modelli sia tramite k-fold che altri metodi

- Per prima cosa definiamo la metodologia di cross-validation da applicare
- Usiamo ad esempio un oggetto della classe `KFold`
  - il primo parametro è il numero di fold (k) da usare, prendiamo 5 come esempio
  - specifichiamo inoltre che i dati sono distribuiti casualmente e il seed da usare

In [62]:
from sklearn.model_selection import KFold
kf = KFold(5, shuffle=True, random_state=42)

- Definiamo un modello da validare, ad es. il modello kernel ridge visto sopra

In [63]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="poly", degree=5))
])

- Per eseguire la CV usiamo quindi la funzione `cross_validate`, a cui passiamo in input:
  - la definizione di un modello, di cui viene addestrata una copia con la stessa configurazione per ciascun fold
  - i dati, divisi come per `fit` in valori di variabili indipendenti (X) e dipendente (y)
  - l'oggetto che definisce la strategia di CV, in questo caso l'istanza di `KFold`
  - l'opzione `return_train_score=True` per ottenere la valutazione di ogni modello anche sui dati di addestramento

In [64]:
from sklearn.model_selection import cross_validate
cv_result = cross_validate(model, X, y, cv=kf, return_train_score=True)

- Otteniamo un dizionario con un vettore per ciascuna misura estratta, ciascuno ha un valore per ognuno dei 5 fold

In [65]:
cv_result

{'fit_time': array([0.03852463, 0.03081799, 0.03089452, 0.0287087 , 0.0285511 ]),
 'score_time': array([0.0121448 , 0.01262403, 0.00940895, 0.00907612, 0.00944591]),
 'test_score': array([0.79093529, 0.83709349, 0.69734727, 0.90845112, 0.7917183 ]),
 'train_score': array([0.91333254, 0.90350452, 0.90330007, 0.8981712 , 0.90764729])}

- Per operare facilmente, raccogliamo i dati in un `DataFrame`

In [66]:
cv_table = pd.DataFrame(cv_result)
cv_table

,fit_time,score_time,test_score,train_score
0,0.038525,0.012145,0.790935,0.913333
1,0.030818,0.012624,0.837093,0.903505
2,0.030895,0.009409,0.697347,0.903300
3,0.028709,0.009076,0.908451,0.898171
4,0.028551,0.009446,0.791718,0.907647


- Per ognuno dei 5 fold vediamo riportati
  - i secondi impiegati per l'addestramento (`fit_time`) e la validazione (`score_time`) del modello
  - lo score calcolato su training set (`train_score`) e validation set (`test_score`)
- Lo score è quello calcolato dal metodo `score` del modello, ovvero il coefficiente R²

- Per avere un dato generale sulla bontà del modello, possiamo calcolare media e deviazione standard degli score

In [67]:
cv_table.agg(["mean", "std"])

,fit_time,score_time,test_score,train_score
mean,0.031499,0.010540,0.805109,0.905191
std,0.004082,0.001698,0.076967,0.005657


- Tale valutazione è più affidabile di quella col metodo hold-out, ottenuta da un singolo modello
- Ci permette inoltre di valutare la "robustezza" del modello, ovvero quanto l'accuratezza sia stabile addestrandosi su set di dati diversi

## Esercizio 3: Cross-validation su modello kernel ridge

- **(3a)** Definire un modello di regressione kernel ridge su feature standardizzate con kernel polinomiale di 3° grado e $\alpha=10$
- **(3b)** Eseguire la cross-validation del modello, utilizzando 5 fold dell'intero set di dati (`X` e `y`) generati dall'oggetto `kf` definito sopra
- **(3c)** Calcolare la media e la deviazione standard dei punteggi R² ottenuti dalla validazione di ciascun fold

## Ricerca degli iperparametri con grid search

- Sui modelli utilizzati finora abbiamo impostato manualmente i valori di diversi iperparametri
  - grado della regressione polinomiale, peso della regolarizzazione, ...
- L'accuratezza del modello può dipendere fortemente da questi valori
- Scelto un generico modello da utilizzare (es. regressione polinomiale o kernel ridge), vorremmo **individuare i valori degli iperparametri che ne massimizzino l'accuratezza**
- scikit-learn fornisce un supporto per effettuare una ricerca "a griglia" (_grid search_) sui valori degli iperparametri, effettuando la cross validation di un modello con diverse configurazioni

- Consideriamo ad esempio un modello _elastic net_ di cui fissiamo arbitrariamente il parametro `l1_ratio`

In [72]:
model = ElasticNet(l1_ratio=0.2)

- Vorremmo trovare il migliore valore possibile del parametro `alpha` tra un insieme di valori possibili, ad esempio:

In [73]:
candidate_alphas = [  0.01,    0.1,     1,    10]
# ovvero
candidate_alphas = [10**-2, 10**-1, 10**0, 10**1]
# che si può ottenere con
candidate_alphas = np.logspace(-2, 1, 4)

- Creiamo ora la _griglia_ dei parametri, ovvero un dizionario in cui associamo ai nomi dei parametri variabili i valori che possono assumere
- In questo caso abbiamo un unico parametro variabile, `alpha`

In [74]:
grid = {"alpha": candidate_alphas}

- Definiamo ora un modello `GridSearchCV` indicando
  - il modello "base" con i parametri fissati a priori
  - la griglia dei parametri variabili
  - il metodo `cv` di cross validation da usare

In [75]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(model, grid, cv=kf)

- Come per i modelli base, usiamo il metodo `fit` per eseguire l'addestramento, passando la matrice X e il vettore y
- Per ogni valore possibile di `alpha`, scikit-learn esegue la cross-validation per calcolare il punteggio R² medio del modello con quel valore di `alpha`

In [76]:
gs.fit(X_train, y_train);

- In seguito ai test, il modello impostato viene (di default) riaddestrato su tutti i dati forniti, usando gli iperparametri che han dato il miglior punteggio medio
- Il modello finale è accessibile all'attributo `gs_best_estimator_`

In [77]:
gs.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=0.2)

- Dall'attributo `best_params_` possiamo vedere quali sono i valori selezionati dalla griglia degli iperparametri per tale modello

In [78]:
gs.best_params_

{'alpha': 0.1}

- L'oggetto `GridSearchCV` addestrato può essere usato come un normale modello di predizione, le chiamate a `predict` e altri metodi sono girate al `best_estimator_`

In [79]:
# prezzo predetto per la prima riga del validation set
gs.predict(X_val.iloc[[0]])

array([28.35759953])

In [80]:
# equivalente a
gs.best_estimator_.predict(X_val.iloc[[0]])

array([28.35759953])

- Possiamo valutare il modello sul validation set "esterno", non utilizzato nella grid search

In [81]:
print_eval(X_val, y_val, gs)

   Mean squared error: 21.905
       Relative error: 16.80121%
R-squared coefficient: 0.7088


- L'attributo `cv_results_` fornisce risultati dettagliati su tutti gli iperparametri testati
- Come per `cross_validate`, raccogliamo i risultati in un `DataFrame` per visualizzarli meglio

In [82]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005117,0.001650,0.003252,0.000595,0.01,{'alpha': 0.01},0.728071,0.483786,0.645836,0.711862,0.801124,0.674136,0.107231,2
1,0.003867,0.001181,0.002479,0.000790,0.1,{'alpha': 0.1},0.727252,0.496850,0.637157,0.721205,0.800228,0.676539,0.103639,1
2,0.003206,0.000407,0.001909,0.000255,1.0,{'alpha': 1.0},0.714022,0.523966,0.594603,0.670194,0.761741,0.652905,0.084740,3
3,0.002807,0.000233,0.001760,0.000049,10.0,{'alpha': 10.0},0.624769,0.373641,0.572294,0.539693,0.623149,0.546709,0.092296,4


I dati riportati per ciascun test includono:
- `{mean|std}_{fit|score}_time`: media/dev. standard dei tempi di addestramento/valutazione sui diversi fold
- `param_X`: valore del parametro X
- `params`: dizionario col valore di tutti i parametri
- `splitN_test_score`: punteggio della valutazione sull'N-esimo fold
- `{mean|std}_test_score`: media/dev. standard dei punteggi sui diversi fold
- `rank_test_score`: ranking del punteggio, 1 è il migliore

- Cosa succede con più iperparametri variabili?
- Impostiamo ad esempio sia 3 valori possibili per `alpha` che 3 valori possibili per `l1_ratio`

In [83]:
model = ElasticNet()
grid = {
    "alpha":    [0.1, 1, 10],
    "l1_ratio": [0.1, 0.2, 0.3]
}
gs = GridSearchCV(model, grid, cv=kf)
gs.fit(X_train, y_train);

- Visualizzo i risultati ordinati per punteggio R² medio decrescente

In [84]:
pd.DataFrame(gs.cv_results_).sort_values("mean_test_score", ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005770,0.000322,0.003815,0.000422,0.1,0.1,"{'alpha': 0.1, 'l1_ratio': 0.1}",0.727791,0.499753,0.636757,0.721171,0.800223,0.677139,0.102714,1
1,0.005260,0.000362,0.003425,0.000808,0.1,0.2,"{'alpha': 0.1, 'l1_ratio': 0.2}",0.727252,0.496850,0.637157,0.721205,0.800228,0.676539,0.103639,2
2,0.003544,0.000410,0.001974,0.000356,0.1,0.3,"{'alpha': 0.1, 'l1_ratio': 0.3}",0.726624,0.493642,0.637503,0.721081,0.800186,0.675807,0.104646,3
3,0.002546,0.000056,0.001573,0.000081,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",0.714960,0.526437,0.595148,0.670794,0.763423,0.654152,0.084507,4
4,0.002571,0.000069,0.001557,0.000045,1,0.2,"{'alpha': 1, 'l1_ratio': 0.2}",0.714022,0.523966,0.594603,0.670194,0.761741,0.652905,0.084740,5
5,0.002503,0.000042,0.001578,0.000099,1,0.3,"{'alpha': 1, 'l1_ratio': 0.3}",0.714237,0.521933,0.594699,0.669538,0.759777,0.652037,0.084850,6
6,0.002510,0.000105,0.001670,0.000193,10,0.1,"{'alpha': 10, 'l1_ratio': 0.1}",0.638007,0.397883,0.577252,0.557198,0.655982,0.565264,0.091374,7
7,0.002620,0.000114,0.001590,0.000095,10,0.2,"{'alpha': 10, 'l1_ratio': 0.2}",0.624769,0.373641,0.572294,0.539693,0.623149,0.546709,0.092296,8
8,0.002507,0.000098,0.001588,0.000053,10,0.3,"{'alpha': 10, 'l1_ratio': 0.3}",0.614417,0.357906,0.570172,0.529856,0.607627,0.535995,0.094024,9


- scikit-learn ha generato e testato **tutte le combinazioni possibili** dei valori degli iperparametri, in tutto 3×3 = 9 configurazioni

### Grid search su pipeline

- Possiamo usare `GridSearchCV` anche con una pipeline, testando diversi valori per i parametri dei singoli componenti (filtri e modello)
- Consideriamo ad esempio un modello polinomiale con regolarizzazione L2, su cui sono variabili
  - il grado del polinomio (attributo `degree` del filtro `poly`)
  - il peso della regolarizzazione (attributo `alpha` del modello `regr`)

In [85]:
model = Pipeline([
    ("poly",  PolynomialFeatures(include_bias=False)),
    ("scale", StandardScaler()),
    ("regr",  Ridge())
])

- Per riferirsi ai parametri dei singoli componenti, usiamo la notazione `componente__parametro` _(con DUE underscore in mezzo)_

In [86]:
grid = {
    "poly__degree": [2, 3],      # <- grado polinomio
    "regr__alpha":  [0.1, 1, 10] # <- regolarizzazione
}

- Il resto del funzionamento rimane invariato

In [87]:
gs = GridSearchCV(model, grid, cv=kf)
gs.fit(X_train, y_train);
pd.DataFrame(gs.cv_results_).sort_values("mean_test_score", ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_poly__degree,param_regr__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.025326,0.003006,0.004403,0.001998,2,1,"{'poly__degree': 2, 'regr__alpha': 1}",0.835447,0.596491,0.802240,0.810611,0.882463,0.785450,0.098521,1
2,0.030462,0.003201,0.004411,0.002281,2,10,"{'poly__degree': 2, 'regr__alpha': 10}",0.842239,0.594859,0.750900,0.795789,0.856246,0.768007,0.094172,2
0,0.030712,0.004690,0.004241,0.001379,2,0.1,"{'poly__degree': 2, 'regr__alpha': 0.1}",0.824409,0.648065,0.850516,0.816894,0.493138,0.726604,0.136929,3
5,0.031003,0.003968,0.006002,0.001280,3,10,"{'poly__degree': 3, 'regr__alpha': 10}",0.853895,0.454814,0.773790,0.748621,0.333358,0.632895,0.201753,4
4,0.029644,0.001920,0.006447,0.000761,3,1,"{'poly__degree': 3, 'regr__alpha': 1}",0.852441,0.568150,0.794898,0.775824,-3.047350,-0.011207,1.521113,5
3,0.030254,0.001244,0.006951,0.000270,3,0.1,"{'poly__degree': 3, 'regr__alpha': 0.1}",0.692042,0.664729,0.648950,0.784034,-3.348237,-0.111696,1.618947,6


- Nella pipeline possiamo inoltre impostare un intero componente come parametro variabile, con la possibilità di rimuoverlo impostandolo a `None`
- Possiamo ad esempio testare un modello con e senza standardizzazione delle feature

In [88]:
model = Pipeline([
    ("scale", None),   # uso None come segnaposto
    ("regr",  Ridge())
])
grid = {
    # scale = standardizzazione oppure nulla
    "scale": [None, StandardScaler()],
    "regr__alpha": [0.1, 1, 10]
}
gs = GridSearchCV(model, grid, cv=kf)
gs.fit(X_train, y_train);
pd.DataFrame(gs.cv_results_).sort_values("mean_test_score", ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regr__alpha,param_scale,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.005952,0.000248,0.002513,0.000128,1,StandardScaler(),"{'regr__alpha': 1, 'scale': StandardScaler()}",0.735767,0.495302,0.657167,0.720101,0.800925,0.681853,0.103883,1
1,0.006078,0.000081,0.002504,0.000084,0.1,StandardScaler(),"{'regr__alpha': 0.1, 'scale': StandardScaler()}",0.735121,0.495993,0.656523,0.720110,0.801136,0.681776,0.103648,2
5,0.005859,0.000128,0.002440,0.000123,10,StandardScaler(),"{'regr__alpha': 10, 'scale': StandardScaler()}",0.741449,0.487396,0.662300,0.718528,0.798668,0.681668,0.106525,3
0,0.007792,0.003227,0.003392,0.001026,0.1,None,"{'regr__alpha': 0.1, 'scale': None}",0.734675,0.494498,0.655446,0.718577,0.801852,0.681009,0.104244,4
2,0.003613,0.000091,0.002297,0.000081,1,None,"{'regr__alpha': 1, 'scale': None}",0.730686,0.487010,0.649269,0.712555,0.801934,0.676291,0.106412,5
4,0.003588,0.000105,0.002234,0.000111,10,None,"{'regr__alpha': 10, 'scale': None}",0.726584,0.488528,0.641675,0.719819,0.801424,0.675606,0.106333,6


## Esercizio 4: Grid search su regressione kernel ridge

- **(4a)** Creare una funzione `grid_test` che, presi in input un modello e una griglia di parametri
  - esegua una grid search con `X_train`, `y_train` come dati di addestramento con modello e parametri forniti, usando cross-validation a 5 fold
  - stampi (`print(...)`) il dizionario con gli iperparametri selezionati del modello migliore
  - stampi le misure di accuratezza (`print_eval`) sui dati `X_val`, `y_val`
- **(4b)** Testare con la funzione sopra un modello di regressione elastic net polinomiale con
  - grado 2 o 3
  - standardizzazione delle feature generate
  - `alpha` pari a 0.1, 1 o 10
  - `l1_ratio` pari a 0.1, 0.25 o 0.5
- **(4c)** Testare con la funzione sopra un modello kernel ridge con
  - standardizzazione dei dati
  - kernel polinomiale di grado compreso tra 2 e 10
  - `alpha` (regolarizzazione) pari a 0.01, 0.1, 1 o 10

In [90]:
def grid_test(model, grid):
    ...

## Nested cross-validation

- Negli esercizi sopra abbiamo validato il risultato finale della grid search su un validation set separato
- In altre parole, abbiamo usato il metodo k-fold per l'ottimizzazione degli iperparametri, mentre abbiamo usato hold-out per la validazione finale
- La _nested k-fold cross-validation_ prevede che siano generati **k fold "esterni"** su tutti i dati disponibili e che **per ciascuno** si esegua il tuning degli iperparametri con una cross validation "interna" usando le parti di training dei fold esterni
- I metodi di cross validation esterna ed interna possono differire, ad es. avendo un diverso numero di fold
- Ipotizziamo ad esempio di usare 3 fold esterni e 5 interni

In [94]:
outer_cv = KFold(3, shuffle=True, random_state=42)
inner_cv = KFold(5, shuffle=True, random_state=42)

## Esercizio 5: Nested cross validation

- **(5a)** Completare l'implementazione una funzione `nested_cv` che esegua la nested cross validation
  - predisporre una lista vuota in cui salvare i punteggi ottenuti su ogni fold esterno
  - usare un ciclo `for` per iterare tutti i fold esterni (T, V) del dataset `X`, `y`
    - usare il metodo `split` di `outer_cv`, che fornisce per ogni fold gli indici delle istanze di training e di validation
  - su ciascun fold esterno, eseguire la grid search con modello e parametri dati in ingresso alla funzione sui dati T, applicando `inner_cv` come cross validation
  - per ogni grid search, salvare nella lista di punteggi il R² ottenuto dalla validazione del modello finale sui dati V
  - restituire la lista alla fine del ciclo
- **(5b)** Testare la funzione su uno dei modelli usati sopra

In [96]:
def nested_cv(model, grid):
    results = []
    for train_indices, val_indices in outer_cv.split(X, y):
        X_train, y_train = X.iloc[train_indices], y.iloc[train_indices]
        X_val, y_val = X.iloc[val_indices], y.iloc[val_indices]
        ...
    return results

## Appendice: Disabilitare i warning

- Alcuni dei modelli utilizzati possono mostrare dei warning nel caso l'addestramento non riesca a convergere
  - questi warning possono essere ripetuti più volte durante cross validation e grid search
- Eseguire il seguente codice per non stampare questa categoria di warning (adattare per altre simili, ad es. `DeprecationWarning`):

In [99]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)